<a href="https://colab.research.google.com/github/Stella-Achar-Oiro/NLP-Sentinel-Analysis/blob/main/Streamlit_App_for_Covid_19_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#SETUP GPU
import tensorflow as tf
device_list = tf.test.gpu_device_name
print("Tensorflow version:", tf.__version__)

if tf.test.gpu_device_name():
    print("GPU is available")
else:
    print("GPU is NOT available")
device_list()

Tensorflow version: 2.12.0
GPU is available


'/device:GPU:0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 117.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=2ce59691bea11029e11ee00e6c48a0f8c15a33edbf7b438df323628126f2cdde
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [4]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.8 MB/s eta 0:00:00


In [5]:
import streamlit as st 
import numpy as np
import transformers
import torch
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import select
from PIL import Image



# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/P5/eval_subset.csv')
st.set_page_config(
    page_title="Twitter Sentiment Analyzer", page_icon="📊", layout="wide"
)

#Image Description
image = Image.open('/content/drive/MyDrive/P5/tweet analysis.jpg')
image_y = Image.open('/content/drive/MyDrive/P5/twitter bird.jpg')
st.image([image,image_y])


@st.cache_resource
def get_model():
    model = transformers.AutoModelForSequenceClassification.from_pretrained("Achar/NLP-Sentimental-Analysis-using-DistilBERT-ModeL")
    tokenizer = transformers.AutoTokenizer.from_pretrained("Achar/NLP-Sentimental-Analysis-using-DistilBERT-ModeL")
    return tokenizer,model

tokenizer, model = get_model()

st.header("Covid19 Twitter Sentimental Analysis")
st.text("This app uses Distilbert-base-uncased for the analysis.")
add_text_sidebar = st.sidebar.title("Menu")
with st.sidebar:
    st.title("Twitter Sentiment Analyzer")

    st.markdown(
        """
        <div style="text-align: justify;">
            This app performs sentiment analysis on the latest tweets based on 
            the entered search term. Since the app can only predict positive or 
            negative, and neutral sentiment, it is more suitable towards analyzing the 
            sentiment of brand, product, service, company, or person. 
            Only English tweets are supported.
        </div>
        """,
        unsafe_allow_html=True,
    )

#Graphical representation
st.sidebar.markdown("### Number of tweets by sentiment")
sentiment_count = df['safe_text'].value_counts()
sentiment_count = pd.DataFrame({'Sentiment':sentiment_count.index, 'Tweets':sentiment_count.values})
if not st.sidebar.checkbox("Hide", True):
    st.markdown("### Number of tweets by sentiment")
    if select == 'Bar plot':
        fig = px.bar(sentiment_count, x='Sentiment', y='Tweets', color='Tweets', height=50)
        st.plotly_chart(fig)
    else:
        fig = px.pie(sentiment_count, values='Tweets', names='Sentiment')
        st.plotly_chart(fig)





# Side view description 
st.sidebar.subheader("Show random tweet")
random_tweet = st.sidebar.radio('Sentiment', ('positive', 'neutral', 'negative'))
st.sidebar.markdown(df['safe_text'])


st.markdown("[Github link](https://github.com/Stella-Achar-Oiro)")
st.markdown("[LinkedIn link](https://www.linkedin.com/in/stella-achar-oiro/)")
st.markdown('Created by Foster,Kwasi,Linda,Stella,Joshua and Bright')
user_input = st.text_area('Enter text to predict')
button = st.button('predict')


# Define Helper Function
label = {0: 'Negative', 1:'Neutral', 2:'Positive'}


      
# Prediction
if user_input and button:
    test_input = tokenizer([user_input],return_tensors='pt')
    st.slider("Number of tweets", min_value=100, max_value=2000, key="user_input")

    # Test output
    output = model(**test_input)
    st.write('Logits:',output.logits)
    predicted_class = np.argmax(output.logits.detach().numpy())
    st.write('prediction:',label[predicted_class])

2023-05-15 20:17:20.105 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]
